In [1]:
import numpy as np
import open3d as o3d
from tqdm import tqdm_notebook
import os, sys, shutil
import matplotlib.pyplot as plt

In [2]:
BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/"

FOLDER = "d"
OUTPUT_FOLDER = "d_img_40_512"
IS_MASK = False

# STATIC_PATH  = os.path.join(BASE_PATH, STATIC_FOLDER)
FOLDER_PATH = os.path.join(BASE_PATH, FOLDER)
OUTPUT_FOLDER_PATH = os.path.join(BASE_PATH, OUTPUT_FOLDER)
if not os.path.exists(OUTPUT_FOLDER_PATH):
    os.makedirs(OUTPUT_FOLDER_PATH)
else:
    shutil.rmtree(OUTPUT_FOLDER_PATH)
    os.makedirs(OUTPUT_FOLDER_PATH)

In [3]:
def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

def linear_transform(sample_img, A=-1, B=1, a=0, b=255):
    mul_fact = (b-a)/(B-A)
    sub_fact = A
    add_fact = a
    out_img = (sample_img-sub_fact)*mul_fact + add_fact
    return out_img

def get_rgb_from_img(img, is_mask=IS_MASK):
    shp = img.shape
    if not is_mask:
        img = img.reshape(1, shp[0], shp[1], shp[2])
        frame_untransformed = from_polar_np(img)[0]
    else:
        replicate = img[1].reshape(1,shp[1], shp[2]) # dim 1 is dynamic in the mask npy
        frame_untransformed = np.concatenate([replicate, replicate, replicate])
        
    frame = linear_transform(frame_untransformed)
    frame_img = frame.transpose(1,2,0).astype("uint8")
    return frame_img

def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()
        
def getint(name):
    try:
        return int(name.split('.')[0])
    except Exception as e:
        print("Error occured while trying to read {}".format(name))
    return None

In [4]:

for npy_file in sorted(os.listdir(FOLDER_PATH), key=getint):
    print(npy_file)
    npy_path = os.path.join(FOLDER_PATH, npy_file)
    npy_arr = np.load(npy_path)
    
    file_idx = 1
    OUTPUT_sub_folder_PATH = os.path.join(OUTPUT_FOLDER_PATH, npy_file.split(".")[0])
    if not os.path.exists(OUTPUT_sub_folder_PATH):
        os.makedirs(OUTPUT_sub_folder_PATH)
    else:
        shutil.rmtree(OUTPUT_sub_folder_PATH)
        os.makedirs(OUTPUT_sub_folder_PATH)
    
    for arr in tqdm_notebook(npy_arr):
        this_arr = get_rgb_from_img(arr[:,5:45,::2])
        
        png_file = str(file_idx) + ".png"
        png_path = os.path.join(OUTPUT_sub_folder_PATH, png_file)
        plt.imsave(png_path, this_arr)
        
        file_idx += 1

Error occured while trying to read emptycities_testd15.npy
emptycities_testd15.npy


/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [5]:
this_arr.shape

(40, 512, 3)